**Importing Necessary Libraries**

In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from tabulate import tabulate

**Node**

In [2]:
class Node():
    def __init__(self, left=None, right=None, threshold=None, feature_index=None, information_gain=None, value=None):
        """
        Initialize a Node object for a decision tree.
        """
        self.left = left             # Set the left child node.
        self.right = right             # Set the right child node.
        self.threshold = threshold       # Set the threshold value for splitting at this node.
        self.feature_index = feature_index    # Set the index of the feature used for splitting at this node.
        self.information_gain = information_gain   # Set the information gain of splitting at this node.
        self.value = value                       # Set the predicted value at this node (for leaf nodes).

**Decision Tree**

In [3]:
class DecisionTree():
    def __init__(self,min_samples_split=2):
        """Initialize a DecisionTree object with the specified minimum number of samples required to split a node.
        """
        self.root = None                                  # Initialize the root node as None.
        self.min_samples_split = min_samples_split        # Set the minimum number of samples required to split a node.

    
    def fit(self,X,y):
        """Fit the decision tree to the training data."""
        
        df = np.concatenate((X,y), axis = 1)     # Concatenate the features and labels to create the dataset.
        self.root = self.grow_tree(df)         # Build the decision tree using the dataset.

        
    def grow_tree(self,df):
        """Recursive function to build the decision tree."""
        X = df[:,:-1]       # Extract features from the dataframe.
        y = df[:,-1]          # Extract labels from the dataframe.
        n_samples, n_features = X.shape   # Get the number of samples and features in the dataset.

        if (n_samples > self.min_samples_split):
            best_one = self.best_split(df,n_samples,n_features)      # Find the best split for the current node.

            if (best_one and best_one['information_gain'] > 0):

                left_tree = self.grow_tree(best_one['left_part'])     # Recursively build the left subtree.

                right_tree = self.grow_tree(best_one['right_part'])       # Recursively build the right subtree.
                return Node(left_tree, right_tree, best_one['threshold'], best_one['feature_index'], best_one['information_gain']) # Create a decision node.

        leaf_value = self.leaf_val(y)      # Else, Determine the leaf value for the current node.

        return Node(value = leaf_value)      # create a leaf node with the determined value.

    
    def leaf_val(self,y):
        """Determine the most common label in a set of labels as the leaf value."""

        return max(list(y), key = list(y).count)

    
    def best_split(self, df, n_samples, n_features):
        """Find the best split for the current node."""
        best_one = {}                # Initialize a dictionary to store information about the best split.
        
        max_ig = -float('inf')          # Initialize the maximum information gain to negative infinity.
        
        for index in range(n_features):
            
            other_val = df[:,index]        # Get the values of the current feature.

            vals = set(other_val)          # Get the unique values of the current feature.

            for thresh in vals:

                left_split,right_split = self.custom_split(df,thresh,index)   # Split the dataset based on the current feature and threshold.

                if (len(left_split) > 0 and len(right_split) > 0):
                    
                    y, y_left, y_right = df[:,-1], left_split[:,-1], right_split[:,-1]       # Extract labels for the splits.
                    
                    inform_gain = self.calculate_information_gain(y, y_left, y_right)     # Calculate the information gain for the current split.
              
                    if (inform_gain > max_ig):

                        # Update the best split information.
                        best_one['feature_index'] = index 
                        best_one['information_gain'] = inform_gain
                        best_one['threshold'] = thresh
                        best_one['left_part'] = left_split
                        best_one['right_part'] = right_split

                        max_ig = inform_gain       # Update the maximum information gain.

        return best_one         # Return the information about the best split.
 
    
    def custom_split(self, df, thresh, index):
        """Split the dataset based on a given threshold value for a specific feature."""
        
        feat_val = df[:, index]                   # Get the values of the specified feature.
        left_side = df[feat_val <= thresh]        # Create a split for values less than or equal to the threshold.
        right_side = df[feat_val > thresh]        # Create a split for values greater than the threshold.
        return left_side, right_side              


    def entropy(self,y):
        """ Calculate the entropy of a set of labels."""
        
        _, counts = np.unique(y, return_counts=True)               # Get the counts of each unique label.
        probabilities = counts / len(y)                              # Calculate the probabilities of each label.
        entropy = -np.sum(probabilities * np.log2(probabilities))      # Calculate the entropy.
        return entropy
    

    def calculate_information_gain(self, parent_labels, left_labels, right_labels):
        """Calculate the information gain of splitting a set of labels into two subsets."""

        p_left = len(left_labels) / len(parent_labels)             # Calculate the proportion of samples in the left subset.
        p_right = len(right_labels) / len(parent_labels)           # Calculate the proportion of samples in the right subset.
        
         # Calculate the information gain.
        infogain = self.entropy(parent_labels) - (p_left * self.entropy(left_labels) + p_right * self.entropy(right_labels))
        return infogain

    def predict(self,X):
        """Make predictions for a set of instances."""
        
        return [self.predict_instance(x,self.root) for x in X]  # Make predictions for each instance.

    def predict_instance(self,x, node):
        """Recursively traverse the decision tree to make a prediction for a single instance."""
        
        if (node.value != None):                          # Check if the current node is a leaf node.
            return node.value

        if (x[node.feature_index] <= node.threshold):       # Check if the instance should go to the left subtree.
            return self.predict_instance(x,node.left)       # Recursively traverse the left subtree.
        else:
            return self.predict_instance(x,node.right)      # Recursively traverse the left subtree.


**For the Iris dataset, calculating the accuracy using 10 fold cross-validation for each value of min.(n_min E {5, 10, 15, 20})**

In [4]:
def cross_validate_iris(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Define the values for min_samples_split
    n_min_values = [5, 10, 15, 20]
    
    # Number of splits for KFold
    num_splits = 10
    
    # Initialize KFold with specified number of splits
    kf = KFold(n_splits=num_splits)
    
    # Create a matrix to store results (accuracy scores)
    result_matrix = np.zeros((len(n_min_values), num_splits))
    
    # Iterate over each min_samples_split value
    for i, n_min in enumerate(n_min_values):
        
        # Iterate over each split in KFold
        for index, (train_index, _) in enumerate(kf.split(X_train, y_train)):
            
            # Get the training data for the current split
            x_train_fold, y_train_fold = X_train[train_index], y_train[train_index]
            
            # Initialize a decision tree classifier with current min_samples_split value
            decision_tree = DecisionTree(min_samples_split=n_min / 100 * len(x_train_fold))
            
            # Fit the classifier on the training data
            decision_tree.fit(x_train_fold, y_train_fold)
            
            # Make predictions on the test set
            y_pred = decision_tree.predict(X_test)
            
            # Calculate and store the accuracy score for the current split
            result_matrix[i][index] = accuracy_score(list(y_test.reshape(len(y_pred),)), y_pred)
        
        # Calculate average accuracy and standard deviation for the current min_samples_split value
        avg_accuracy = np.mean(result_matrix[i, :])
        avg_std_dev = np.std(result_matrix[i, :])
        
        # Print the average accuracy and standard deviation for the current min_samples_split value
        print("For n_min =", n_min, ":\n", 
              "average accuracy:", avg_accuracy, "\n",
              "average standard deviation:", avg_std_dev, "\n")

    # Print results in a table
    headers = ["n_min", "Average Accuracy", "Average Standard Deviation"]
    data = [[n_min_values[i], np.mean(result_matrix[i, :]), np.std(result_matrix[i, :])] for i in range(len(n_min_values))]
    print(tabulate(data, headers = headers, tablefmt = "grid"))



In [5]:
df = pd.read_csv("./iris.csv", names = ['Length_Sepal', 'Width_Sepal', 'Length_Petal','Width_Petal', 'Target'])
df = df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2})


X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values.reshape(-1,1)
cross_validate_iris(X,y)

For n_min = 5 :
 average accuracy: 0.97 
 average standard deviation: 0.03480102169636849 

For n_min = 10 :
 average accuracy: 0.9666666666666666 
 average standard deviation: 0.02108185106778919 

For n_min = 15 :
 average accuracy: 0.9733333333333334 
 average standard deviation: 0.019999999999999997 

For n_min = 20 :
 average accuracy: 0.9733333333333334 
 average standard deviation: 0.019999999999999997 

+---------+--------------------+------------------------------+
|   n_min |   Average Accuracy |   Average Standard Deviation |
+=========+====================+==============================+
|       5 |           0.97     |                    0.034801  |
+---------+--------------------+------------------------------+
|      10 |           0.966667 |                    0.0210819 |
+---------+--------------------+------------------------------+
|      15 |           0.973333 |                    0.02      |
+---------+--------------------+------------------------------+
|      20

**For the Spambase dataset, calculating the accuracy using 10 fold cross-validation for each value of min (n_min E {5, 10, 15, 20, 25})**

In [6]:
def cross_validate_spambase(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)
    
    # Define the values for min_samples_split
    n_min_values = [5, 10, 15, 20, 25]
    
    # Number of splits for KFold
    num_splits = 10
    
    # Initialize KFold with specified number of splits
    kf = KFold(n_splits=num_splits)
    
    # Create a matrix to store results (accuracy scores)
    result_matrix = np.zeros((len(n_min_values), num_splits))
    
    # Iterate over each min_samples_split value
    for i, n_min in enumerate(n_min_values):
        
        # Iterate over each split in KFold
        for index, (train_index, _) in enumerate(kf.split(X_train, y_train)):
            
            # Get the training data for the current split
            x_train_fold, y_train_fold = X_train[train_index], y_train[train_index]
            
            # Initialize a decision tree classifier with current min_samples_split value
            my_classifier_model = DecisionTree(min_samples_split = n_min / 100 * len(x_train_fold))
            
            # Fit the classifier on the training data
            my_classifier_model.fit(x_train_fold, y_train_fold)
            
            # Make predictions on the test set
            y_pred = my_classifier_model.predict(X_test)
            
            # Calculate and store the accuracy score for the current split
            result_matrix[i][index] = accuracy_score(list(y_test.reshape(len(y_pred),)), y_pred)
        
        # Calculate average accuracy and standard deviation for the current min_samples_split value
        avg_accuracy = np.mean(result_matrix[i, :])
        avg_std_dev = np.std(result_matrix[i, :])
        
        # Print the average accuracy and standard deviation for the current min_samples_split value
        print("For n_min =", n_min, ":\n", 
              "average accuracy:", avg_accuracy, "\n",
              "average standard deviation:", avg_std_dev, "\n")

    # Print results in a table
    headers = ["n_min", "Average Accuracy", "Average Standard Deviation"]
    data = [[n_min_values[i], np.mean(result_matrix[i, :]), np.std(result_matrix[i, :])] for i in range(len(n_min_values))]
    print(tabulate(data, headers=headers, tablefmt="grid"))


In [7]:
df_spambase = pd.read_csv("./spambase.csv")


X = df_spambase.iloc[:,:-1].values
y = df_spambase.iloc[:,-1].values.reshape(-1,1)


cross_validate_spambase(X,y)

For n_min = 5 :
 average accuracy: 0.9044565217391305 
 average standard deviation: 0.009848209993476704 

For n_min = 10 :
 average accuracy: 0.8891304347826086 
 average standard deviation: 0.002956841414942479 

For n_min = 15 :
 average accuracy: 0.8653260869565218 
 average standard deviation: 0.012000413508943488 

For n_min = 20 :
 average accuracy: 0.8593478260869565 
 average standard deviation: 0.0034441259821206076 

For n_min = 25 :
 average accuracy: 0.8460869565217392 
 average standard deviation: 0.013144536050287784 

+---------+--------------------+------------------------------+
|   n_min |   Average Accuracy |   Average Standard Deviation |
+=========+====================+==============================+
|       5 |           0.904457 |                   0.00984821 |
+---------+--------------------+------------------------------+
|      10 |           0.88913  |                   0.00295684 |
+---------+--------------------+------------------------------+
|      15 | 